In [1]:
import pandas as pd

# 1. 데이터 로드

In [2]:
# 실습 환경은 인코딩 없이 진행 됨
train = pd.read_csv('data/X_train.csv', encoding='euc-kr') 
target = pd.read_csv('data/y_train.csv')
test = pd.read_csv('data/X_test.csv', encoding='euc-kr')

print(train.shape, target.shape, test.shape)

df = train.merge(target, on='cust_id', how='left')
print(df.shape)

(3500, 10) (3500, 2) (2482, 10)
(3500, 11)


# 2. 전처리 & Feature Engineering

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3500 entries, 0 to 3499
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
 10  gender   3500 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 328.1+ KB
None


In [4]:
# 실습 환경에서 desc 짤리기 때문에 출력 추가 코딩 필요
desc = df.describe(include="all") 

for col in df.columns:
    print(col,'\n', desc[col])
    pass

cust_id 
 count     3500.000000
unique            NaN
top               NaN
freq              NaN
mean      1749.500000
std       1010.507298
min          0.000000
25%        874.750000
50%       1749.500000
75%       2624.250000
max       3499.000000
Name: cust_id, dtype: float64
총구매액 
 count     3.500000e+03
unique             NaN
top                NaN
freq               NaN
mean      9.191925e+07
std       1.635065e+08
min      -5.242152e+07
25%       4.747050e+06
50%       2.822270e+07
75%       1.065079e+08
max       2.323180e+09
Name: 총구매액, dtype: float64
최대구매액 
 count     3.500000e+03
unique             NaN
top                NaN
freq               NaN
mean      1.966424e+07
std       3.199235e+07
min      -2.992000e+06
25%       2.875000e+06
50%       9.837000e+06
75%       2.296250e+07
max       7.066290e+08
Name: 최대구매액, dtype: float64
환불금액 
 count     1.205000e+03
unique             NaN
top                NaN
freq               NaN
mean      2.407822e+07
std       4.746453e+

In [5]:
# null - 환불금액
print(sum(df['환불금액'].isna())/df.shape[0])
# 65%가 null min=
print(df['환불금액'].min())
# min 5600.0 이므로, 0으로 대체
df['환불금액'] = df['환불금액'].fillna(0)
print(df.isna().sum().sum())

0.6557142857142857
5600.0
0


In [6]:
# object - 주구매상품, 주구매지점 
#print(df['주구매상품'].value_counts())
#print(df['주구매지점'].value_counts())

# label encoding

from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
df['주구매상품'] = enc.fit_transform(df['주구매상품'])
df['주구매지점'] = enc.fit_transform(df['주구매지점'])

In [7]:
# corr
df.corr()[(abs(df.corr()) > 0.7) & (abs(df.corr()) != 1)]

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
cust_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
총구매액,NaN,NaN,0.70008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
최대구매액,NaN,0.70008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
환불금액,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
주구매상품,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
주구매지점,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
내점일수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
내점당구매건수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
주말방문비율,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
구매주기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# one-hot 
df = pd.get_dummies(df, columns=['주구매상품','주구매지점'], drop_first=True)

# 3. 분류 알고리즘 사용

In [9]:
train.shape[0]*0.7, train.shape[0]*0.8, train.shape[0]

(2450.0, 2800.0, 3500)

In [10]:
# 데이터 분리
from sklearn.model_selection import train_test_split
target = df.gender
df.drop(['cust_id','gender'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2450, 71), (1050, 71), (2450,), (1050,))

# 4. 초매개변수 최적화

# 5. 모형 앙상블

# 6. 예측

# 7. 제출 

In [11]:
# 답안 제출 예시
# 수험번호.csv 생성
# DataFrame.to_csv("0000.csv", index=False)